In [ ]:
# import h5py
# import pandas as pd
# import numpy as np

# # Step 1: Load the data matrix and metadata from the .gctx file
# gctx_file = "../data/raw/level3_beta_ctl_n188708x12328.gctx"
# h5_file = "../data/raw/compound_control_dataset_all.h5"

# chunk_size = 10000  # Number of rows to process at a time

# with h5py.File(gctx_file, "r") as f:
#     # Access data matrix and metadata
#     data = f["/0/DATA/0/matrix"]
#     row_ids = f["/0/META/COL/id"][:].astype(str)  # All sample IDs (rows)
#     col_ids = f["/0/META/ROW/id"][:].astype(int)  # All gene IDs (columns)

#     # Load metadata files
#     gene_metadata = pd.read_csv("../data/raw/geneinfo_beta.txt", sep="\t")
#     pert_info = pd.read_csv("../data/raw/instinfo_beta.txt", sep="\t")

#     # Filter perturbation metadata
#     filtered_pert_info = pert_info[
#         (pert_info["qc_pass"] == 1.0) & (pert_info["pert_type"] == "ctl_vehicle")
#     ].copy()
#     filtered_pert_info.set_index("sample_id", inplace=True)

#     # Filter valid sample IDs
#     valid_sample_ids = pd.Index(row_ids).intersection(filtered_pert_info.index)
#     filtered_pert_info = filtered_pert_info.loc[valid_sample_ids]
#     filtered_pert_info.sort_index(inplace=True)

#     # Create a mapping of sample IDs to row indices
#     row_idx_map = {rid: idx for idx, rid in enumerate(row_ids)}

#     # Compute row indices for valid samples
#     filtered_row_indices = [row_idx_map[sid] for sid in valid_sample_ids]

# # Step 2: Write the filtered data matrix and metadata in chunks
# with h5py.File(h5_file, "w") as h5f:
#     # Pre-allocate space for the dataset in the HDF5 file
#     data_shape = (len(filtered_row_indices), len(col_ids))
#     data_dset = h5f.create_dataset(
#         "data", shape=data_shape, dtype="float32", compression="gzip"
#     )

#     # Write row (sample) metadata
#     for column in filtered_pert_info.columns:
#         h5f.create_dataset(
#             f"row_metadata/{column}",
#             data=filtered_pert_info[column].values.astype("S"),
#         )

#     # Write column (gene) metadata
#     for column in gene_metadata.columns:
#         h5f.create_dataset(
#             f"col_metadata/{column}",
#             data=gene_metadata[column].values.astype("S"),
#         )

#     # Write row and column IDs
#     h5f.create_dataset("row_ids", data=valid_sample_ids.values.astype("S"))
#     gene_mapping = gene_metadata.set_index("gene_id")["gene_symbol"].to_dict()
#     filtered_col_symbols = [gene_mapping.get(gid, f"Gene_{gid}") for gid in col_ids]
#     h5f.create_dataset(
#         "col_ids", data=[x.encode("utf-8") for x in filtered_col_symbols]
#     )

#     # Process and write data in chunks
#     for i in range(0, len(filtered_row_indices), chunk_size):
#         chunk_indices = filtered_row_indices[i : i + chunk_size]  # Current chunk
#         chunk_data = data[chunk_indices, :]  # Load chunk from the data matrix
#         data_dset[i : i + len(chunk_indices), :] = chunk_data  # Write to dataset

# print(f"Filtered data and metadata successfully written to {h5_file}.")

In [ ]:
# import h5py
# import pandas as pd

# # Step 1: Load the data matrix and metadata from the .gctx file
# gctx_file = "../data/raw/level3_beta_ctl_n188708x12328.gctx"

# with h5py.File(gctx_file, "r") as f:
#     # Access the entire data matrix and metadata
#     data = f["/0/DATA/0/matrix"]
#     row_ids = f["/0/META/COL/id"][:].astype(str)  # Sample IDs (rows)
#     col_ids = f["/0/META/ROW/id"][:].astype(int)  # Gene IDs (columns)

#     # Load metadata files
#     gene_metadata = pd.read_csv("../data/raw/geneinfo_beta.txt", sep="\t")
#     pert_info = pd.read_csv("../data/raw/instinfo_beta.txt", sep="\t")

#     # Filter perturbation metadata
#     filtered_pert_info = pert_info[
#         (pert_info["qc_pass"] == 1.0) & (pert_info["pert_type"] == "ctl_vehicle")
#     ].copy()
#     filtered_pert_info.set_index("sample_id", inplace=True)

#     # STEP A: Reduce filtered_pert_info to match row_ids
#     subset_row_ids = pd.Index(row_ids)
#     subset_row_ids = subset_row_ids.intersection(filtered_pert_info.index)
#     subset_pert_info = filtered_pert_info.loc[subset_row_ids].copy()
#     subset_pert_info.sort_index(inplace=True)

#     # Keep a separate column for sample_id (while retaining it as index)
#     subset_pert_info["sample_id"] = subset_pert_info.index

#     # STEP B: Recompute the row indices in the data matrix for valid sample_ids
#     row_idx_map = {rid: idx for idx, rid in enumerate(row_ids)}
#     filtered_row_indices = [row_idx_map[sid] for sid in subset_pert_info.index]

#     # Filter the data matrix to match the final subset of row indices
#     filtered_data_matrix = data[filtered_row_indices, :]

# # Step 2: Map gene IDs to gene symbols
# gene_mapping = gene_metadata.set_index("gene_id")["gene_symbol"].to_dict()
# filtered_col_symbols = [gene_mapping.get(gid, f"Gene_{gid}") for gid in col_ids]

# # Step 3: Write the filtered data and metadata to an HDF5 file
# h5_file = "../data/raw/compound_control_dataset.h5"

# with h5py.File(h5_file, "w") as h5f:
#     # Write the filtered data matrix
#     h5f.create_dataset("data", data=filtered_data_matrix, compression="gzip")

#     # Write row (sample) metadata
#     for column in subset_pert_info.columns:
#         h5f.create_dataset(
#             f"row_metadata/{column}",
#             data=subset_pert_info[column].values.astype("S"),
#         )

#     # Write column (gene) metadata
#     for column in gene_metadata.columns:
#         h5f.create_dataset(
#             f"col_metadata/{column}",
#             data=gene_metadata[column].values.astype("S"),
#         )

#     # Write final row and column IDs
#     h5f.create_dataset("row_ids", data=subset_pert_info.index.values.astype("S"))
#     h5f.create_dataset(
#         "col_ids", data=[x.encode("utf-8") for x in filtered_col_symbols]
#     )

# print(f"Filtered data and metadata successfully written to {h5_file}.")

In [7]:
# h5_file = "../data/raw/compound_control_dataset.h5"
# with h5py.File(h5_file, "r") as h5f:
#     # Load the data matrix
#     data_matrix = h5f["data"][:]

#     # Decode row metadata
#     row_metadata = {
#         key: h5f[f"row_metadata/{key}"][:].astype(str) for key in h5f["row_metadata"]
#     }

#     # Decode column metadata
#     col_metadata = {
#         key: h5f[f"col_metadata/{key}"][:].astype(str) for key in h5f["col_metadata"]
#     }

#     # Load and decode row and column IDs
#     row_ids = h5f["row_ids"][:].astype(str)
#     col_ids = h5f["col_ids"][:].astype(str)

# # Convert metadata to DataFrames for ease of use (optional)
# import pandas as pd

# row_metadata_df = pd.DataFrame(row_metadata)
# col_metadata_df = pd.DataFrame(col_metadata)


In [20]:
# Add src to path
import sys
import os

# Add the src directory to the Python path
sys.path.append(os.path.abspath(os.path.join("..", "src")))

from torch.utils.data import  DataLoader
from data_sets import PertubationDataset
import pandas as pd

controls = "../data/raw/compound_control_dataset.h5"
perturbations = "../data/raw/compound_pertubation_dataset.h5"

smiles_df = pd.read_csv("../data/raw/compoundinfo_beta.txt", sep="\t")
smiles_dict = dict(zip(smiles_df["pert_id"], smiles_df["canonical_smiles"]))

dataset = PertubationDataset(
    controls_file=controls,
    perturbations_file=perturbations,
    smiles_dict=smiles_dict,
    plate_column="det_plate",
    normalize=True,
    n_rows=1000,
)

print(f"Dataset has {len(dataset)} (control, pert) pairs.")

Dataset has 23389 (control, pert) pairs.


In [21]:
# DataLoader
data_loader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=0)

# Example iteration
for batch in data_loader:
    features = batch["features"]
    labels = batch["labels"]
    meta = batch["metadata"]
    print("Features Shape:", features.shape)  # (batch_size, num_genes)
    print("Labels Shape:", labels.shape)  # (batch_size, num_genes)
    print("Metadata sample:", meta["control_metadata"], meta["pert_metadata"])
    break

Features Shape: torch.Size([64, 12328])
Labels Shape: torch.Size([64, 12328])
Metadata sample: {'bead_batch': ['b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15'], 'build_name': ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import string


##############################################################################
# 1) Simple SMILES Encoder
##############################################################################
class SimpleSMILESEncoder(nn.Module):
    """
    A toy SMILES encoder that one-hot encodes each character
    and processes it via a small MLP or CNN. Here we use a trivial MLP
    for demonstration.
    """

    def __init__(self, vocab, embed_dim=64, hidden_dim=128):
        """
        Args:
            vocab (str or list): A string or list of allowed SMILES characters.
            embed_dim (int): Size of the per-character embedding.
            hidden_dim (int): Size of the hidden layer in the MLP that encodes the SMILES.
        """
        super(SimpleSMILESEncoder, self).__init__()
        self.vocab = vocab
        self.vocab_size = len(self.vocab)
        self.embed_dim = embed_dim

        # A simple embedding layer for each character:
        self.embedding = nn.Embedding(self.vocab_size, embed_dim)

        # Then a small MLP to compress all characters into a single vector
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
        )

        # A final pooling transformation to get a single vector
        # (In practice, you might do average/attention pooling across characters.)
        # For simplicity, we apply MLP to each character embedding and average.
        # More advanced: CNN or RNN or Transformer.


    def forward(self, smiles_batch):
        tokens, _ = self.smiles_to_indices(smiles_batch)

        # Make sure tokens are on the same device as the embedding layer
        tokens = tokens.to(self.embedding.weight.device)

        embedded = self.embedding(tokens)  # (B, max_len, embed_dim)
        B, L, E = embedded.shape
        embedded = embedded.view(B * L, E)  # Flatten
        embedded = self.mlp(embedded)  # (B * L, hidden_dim)
        embedded = embedded.view(B, L, -1)  # (B, max_len, hidden_dim)
        embedded = embedded.mean(dim=1)  # (B, hidden_dim)
        return embedded

    def smiles_to_indices(self, smiles_batch):
        """
        Convert list of SMILES strings to a batch of indices for each character.
        We'll simply pad them to max_len in this toy example.
        """
        # Find max length
        max_len = max(len(s) for s in smiles_batch)
        token_ids = []
        lengths = []

        char_to_idx = {c: i for i, c in enumerate(self.vocab)}

        for s in smiles_batch:
            lengths.append(len(s))
            row = []
            for ch in s:
                if ch in char_to_idx:
                    row.append(char_to_idx[ch])
                else:
                    row.append(char_to_idx["?"])  # unknown char
            # pad
            while len(row) < max_len:
                row.append(char_to_idx[" "])  # space or a pad token
            token_ids.append(row)

        # Convert to torch tensor
        tokens = torch.tensor(token_ids, dtype=torch.long)
        return tokens, lengths


##############################################################################
# 2) Multimodal Model: Gene + SMILES
##############################################################################
class MultimodalModel(nn.Module):
    """
    A model that encodes (unperturbed gene expression) + (drug SMILES)
    and predicts (perturbed gene expression).
    """

    def __init__(
        self, gene_dim, gene_hidden_dim=512, drug_hidden_dim=128, smiles_vocab=None
    ):
        """
        Args:
            gene_dim (int): Dimensionality of the gene expression input.
            gene_hidden_dim (int): hidden dimension for the gene MLP.
            drug_hidden_dim (int): hidden dimension for the SMILES encoder output.
            smiles_vocab (str or list): vocabulary for the SMILES.
        """
        super(MultimodalModel, self).__init__()
        if smiles_vocab is None:
            # A simple default vocabulary of typical SMILES chars + space + ?
            # In practice, define a more robust set of tokens
            smiles_vocab = "ACGT()[]=+#@0123456789abcdefghijklmnopqrstuvwxyz" + "? "

        self.gene_dim = gene_dim
        self.gene_hidden_dim = gene_hidden_dim
        self.drug_hidden_dim = drug_hidden_dim

        # 2.1) Gene MLP
        self.gene_encoder = nn.Sequential(
            nn.Linear(gene_dim, gene_hidden_dim),
            nn.ReLU(),
            nn.Linear(gene_hidden_dim, gene_hidden_dim),
            nn.ReLU(),
        )

        # 2.2) SMILES encoder
        self.smiles_encoder = SimpleSMILESEncoder(
            vocab=smiles_vocab, embed_dim=64, hidden_dim=drug_hidden_dim
        )

        # 2.3) Fusion -> final MLP to predict the same dimension as gene_dim
        self.fusion = nn.Sequential(
            nn.Linear(gene_hidden_dim + drug_hidden_dim, gene_hidden_dim),
            nn.ReLU(),
            nn.Linear(
                gene_hidden_dim, gene_dim
            ),  # we predict perturbed expression (size = gene_dim)
        )

    def forward(self, gene_expr, smiles_batch):
        """
        gene_expr: Tensor of shape (B, gene_dim)
        smiles_batch: List of length B containing SMILES strings
        """
        # Encode gene
        gene_emb = self.gene_encoder(gene_expr)  # (B, gene_hidden_dim)
        # Encode SMILES
        drug_emb = self.smiles_encoder(smiles_batch)  # (B, drug_hidden_dim)

        fused = torch.cat(
            [gene_emb, drug_emb], dim=1
        )  # (B, gene_hidden_dim + drug_hidden_dim)
        out = self.fusion(fused)
        return out


##############################################################################
# 3) Training Loop
##############################################################################
def train_multimodal_model(dataset, epochs=5, batch_size=32, lr=1e-3, device="cpu"):
    """
    A training loop for the multimodal model, which uses:
    - gene expression as input
    - SMILES as input
    - predicts perturbed gene expression
    """
    # Infer gene_dim from the first sample
    first_sample = dataset[0]
    gene_dim = first_sample["features"].shape[0]

    # Build a vocab from the first N SMILES in the dataset, or just define one.
    # Here, we rely on the default vocab in MultimodalModel if None is passed.
    model = MultimodalModel(gene_dim=gene_dim).to(device)

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    model.train()
    for epoch in range(epochs):
        total_loss = 0.0
        for batch_data in dataloader:
            # gene expression
            features = batch_data["features"].to(device)  # shape (B, gene_dim)
            labels = batch_data["labels"].to(device)  # shape (B, gene_dim)
            # smiles
            smiles_list = batch_data["smiles"] # a list of SMILES strings of length B

            optimizer.zero_grad()

            outputs = model(features, smiles_list)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * features.size(0)

        avg_loss = total_loss / len(dataset)
        print(f"Epoch [{epoch+1}/{epochs}] - Loss: {avg_loss:.4f}")

    return model

In [23]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = train_multimodal_model(
    dataset, epochs=10, batch_size=64, lr=1e-3, device=device
)
print("Training complete.")

Epoch [1/10] - Loss: 0.0179
Epoch [2/10] - Loss: 0.0114
Epoch [3/10] - Loss: 0.0110
Epoch [4/10] - Loss: 0.0108
Epoch [5/10] - Loss: 0.0106
Epoch [6/10] - Loss: 0.0105
Epoch [7/10] - Loss: 0.0104
Epoch [8/10] - Loss: 0.0164
Epoch [9/10] - Loss: 0.0112
Epoch [10/10] - Loss: 0.0109
Training complete.


In [36]:
# Make model predictions
model.eval()
with torch.no_grad():
    # Get a sample from the dataset
    print(len(dataset))
    sample = dataset[9]
    gene_expr = sample["features"].unsqueeze(0).to(device)
    smiles = sample["smiles"]
    print("Sample SMILES:", smiles)
    pred_expr = model(gene_expr, [smiles]).cpu().numpy()

    print("Baseline Gene Expression:", sample["features"])
    print("Original Gene Expression:", sample["labels"])
    print("Predicted Gene Expression:", pred_expr)

23389
Sample SMILES: CN(C)CC=CC(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2cc1O[C@H]1CCOC1
Baseline Gene Expression: tensor([0.6516, 0.3368, 0.3731,  ..., 0.4273, 0.6222, 0.5413])
Original Gene Expression: tensor([0.6215, 0.3419, 0.2880,  ..., 0.4056, 0.7068, 0.6785])
Predicted Gene Expression: [[0.60999185 0.36232358 0.22071706 ... 0.5611624  0.7454757  0.50741476]]
